In [1]:
from IPython import get_ipython
import datetime
import pandas as pd
import numpy as np
import plotly.express as px
from sodapy import Socrata
import geopandas as gpd
import geopy
dir(geopy)
from geopy.geocoders import Photon
dir(Photon)
from geopy.geocoders import Nominatim
import time
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance
import osmnx as ox
import contextily as ctx
import sys
import json
import folium
dir(folium)

['Choropleth',
 'Circle',
 'CircleMarker',
 'ClickForMarker',
 'ColorLine',
 'ColorMap',
 'CssLink',
 'CustomIcon',
 'Div',
 'DivIcon',
 'Element',
 'FeatureGroup',
 'Figure',
 'FitBounds',
 'GeoJson',
 'GeoJsonPopup',
 'GeoJsonTooltip',
 'Html',
 'IFrame',
 'Icon',
 'JavascriptLink',
 'LatLngPopup',
 'LayerControl',
 'LinearColormap',
 'Link',
 'MacroElement',
 'Map',
 'Marker',
 'PolyLine',
 'Polygon',
 'Popup',
 'Rectangle',
 'RegularPolygonMarker',
 'StepColormap',
 'TileLayer',
 'Tooltip',
 'TopoJson',
 'Vega',
 'VegaLite',
 'WmsTileLayer',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_version',
 'branca',
 'features',
 'folium',
 'map',
 'raster_layers',
 'sys',
 'utilities',
 'vector_layers']

In [2]:
# connect to the data portal
client = Socrata("data.lacity.org", None)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("tkh9-tssh", limit=2000)

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

# print it with .sample, which gives you random rows
df.sample(2)



,zip_code,city,business_name,location_description,street_address,location_end_date,location_start_date,location_account,council_district,location,...,:@computed_region_kqwf_mjcx,:@computed_region_2dna_qi2s,naics,primary_naics_description,:@computed_region_tatf_ua23,:@computed_region_ur2y_g4cx,mailing_city,mailing_zip_code,mailing_address,dba_name
1561,91411-1209,VAN NUYS,VLADIMIR BOGOPOLSKIY,6628 SEPULVEDA 91411-1209,6628 SEPULVEDA BLVD #215,2008-02-11T00:00:00.000,2006-11-01T00:00:00.000,0002246019-0001-2,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528,90048-6201,LOS ANGELES,RITZ CAMERA CENTERS INC,8500 BEVERLY 90048-6201,8500 BEVERLY BLVD #171,2008-01-22T00:00:00.000,1990-02-06T00:00:00.000,0000763836-0001-1,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# add a "where" statement
results = client.get("tkh9-tssh", 
                     limit = 10000, # putting an arbitrary high number (otherwise defaults to 1000)
                     where = "location_end_date between '2020-03-15T00:00:00.000' and '2020-09-30T00:00:00'"
                    )

In [4]:
# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

In [5]:
df.shape


(1977, 16)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1977 entries, 0 to 1976
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   zip_code                   1977 non-null   object
 1   location                   334 non-null    object
 2   mailing_city               195 non-null    object
 3   city                       1977 non-null   object
 4   business_name              1977 non-null   object
 5   naics                      312 non-null    object
 6   primary_naics_description  312 non-null    object
 7   location_description       1977 non-null   object
 8   street_address             1977 non-null   object
 9   location_end_date          1977 non-null   object
 10  mailing_zip_code           196 non-null    object
 11  location_start_date        1977 non-null   object
 12  mailing_address            196 non-null    object
 13  location_account           1977 non-null   object
 14  council_

In [7]:
df.head()

,zip_code,location,mailing_city,city,business_name,naics,primary_naics_description,location_description,street_address,location_end_date,mailing_zip_code,location_start_date,mailing_address,location_account,council_district,dba_name
0,90048-4307,"{'latitude': '34.0719', 'human_address': '{""ad...",WEST HOLLYWOOD,LOS ANGELES,M & S VALET PARKING SERVICES LLC,812930,Parking lots & garages,8022 3RD 90048-4307,8022 W 3RD STREET,2020-03-15T00:00:00.000,90069-4195,2014-04-11T00:00:00.000,1017 N LA CIENEGA BLVD SUITE #200,0002045255-0034-3,5,NaN
1,90010-2830,"{'latitude': '34.0624', 'human_address': '{""ad...",THOUSAND OAKS,LOS ANGELES,WILLIAM JAMES OSBORNE,541100,Legal services,3731 WILSHIRE 90010-2830,3731 WILSHIRE BLVD #514,2020-03-15T00:00:00.000,91320-2151,1998-01-05T00:00:00.000,3155 OLD CONEJO ROAD,0000813748-0001-9,10,NaN
2,90004-0000,NaN,NaN,LOS ANGELES,M & S VALET PARKING SERVICES LLC,NaN,NaN,670 LUCERNE 90004-0000,670 N LUCERNE BLVD,2020-03-15T00:00:00.000,NaN,2016-05-05T00:00:00.000,NaN,0002045255-0059-9,4,NaN
3,90019-6901,NaN,NaN,LOS ANGELES,M & S VALET PARKING SERVICES LLC,NaN,NaN,1048 LA BREA 90019-6901,1048 S LA BREA AVENUE,2020-03-15T00:00:00.000,NaN,2016-05-26T00:00:00.000,NaN,0002045255-0060-2,10,NaN
4,90048-4309,"{'latitude': '34.0721', 'human_address': '{""ad...",WEST HOLLYWOOD,LOS ANGELES,M & S VALET PARKING SERVICES LLC,812930,Parking lots & garages,8108 3RD 90048-4309,8108 W 3RD STREET,2020-03-15T00:00:00.000,90069-4195,2009-12-01T00:00:00.000,1017 N LA CIENEGA BLVD SUITE #200,0002045255-0006-8,5,NaN


In [8]:
#gets rid of floor, apartment, suite, # in address Nominatim and other geocoders won't read it, still has errors, need to join neighborhoods to actual cities
pat = r'^(?P<number>\d+)?(?P<street>.+(?=\bAPT|\bFLOOR|\bBUILDING|\bSUITE|\bUNIT)|.+(?=#)|.+)(?P<apt_unit>(?:\bAPT|\bFLOOR|\bSUITE|\bUNIT|#).+)?'

#adds three columns after running extract
df[['number','street','apt_unit']]=df.street_address.str.extract(pat, expand=True)
df

,zip_code,location,mailing_city,city,business_name,naics,primary_naics_description,location_description,street_address,location_end_date,mailing_zip_code,location_start_date,mailing_address,location_account,council_district,dba_name,number,street,apt_unit
0,90048-4307,"{'latitude': '34.0719', 'human_address': '{""ad...",WEST HOLLYWOOD,LOS ANGELES,M & S VALET PARKING SERVICES LLC,812930,Parking lots & garages,8022 3RD 90048-4307,8022 W 3RD STREET,2020-03-15T00:00:00.000,90069-4195,2014-04-11T00:00:00.000,1017 N LA CIENEGA BLVD SUITE #200,0002045255-0034-3,5,NaN,8022,W 3RD STREET,NaN
1,90010-2830,"{'latitude': '34.0624', 'human_address': '{""ad...",THOUSAND OAKS,LOS ANGELES,WILLIAM JAMES OSBORNE,541100,Legal services,3731 WILSHIRE 90010-2830,3731 WILSHIRE BLVD #514,2020-03-15T00:00:00.000,91320-2151,1998-01-05T00:00:00.000,3155 OLD CONEJO ROAD,0000813748-0001-9,10,NaN,3731,WILSHIRE BLVD,#514
2,90004-0000,NaN,NaN,LOS ANGELES,M & S VALET PARKING SERVICES LLC,NaN,NaN,670 LUCERNE 90004-0000,670 N LUCERNE BLVD,2020-03-15T00:00:00.000,NaN,2016-05-05T00:00:00.000,NaN,0002045255-0059-9,4,NaN,670,N LUCERNE BLVD,NaN
3,90019-6901,NaN,NaN,LOS ANGELES,M & S VALET PARKING SERVICES LLC,NaN,NaN,1048 LA BREA 90019-6901,1048 S LA BREA AVENUE,2020-03-15T00:00:00.000,NaN,2016-05-26T00:00:00.000,NaN,0002045255-0060-2,10,NaN,1048,S LA BREA AVENUE,NaN
4,90048-4309,"{'latitude': '34.0721', 'human_address': '{""ad...",WEST HOLLYWOOD,LOS ANGELES,M & S VALET PARKING SERVICES LLC,812930,Parking lots & garages,8108 3RD 90048-4309,8108 W 3RD STREET,2020-03-15T00:00:00.000,90069-4195,2009-12-01T00:00:00.000,1017 N LA CIENEGA BLVD SUITE #200,0002045255-0006-8,5,NaN,8108,W 3RD STREET,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972,90028-8521,"{'latitude': '34.0983', 'human_address': '{""ad...",DIAMOND BAR,HOLLYWOOD,"OHB RESTAURANT, LLC",722110,Full-service restaurants,5539 SUNSET 90028-8521,5539 W SUNSET BLVD,2020-09-30T00:00:00.000,91765-5499,2019-01-01T00:00:00.000,21700 COPLEY DRIVE SUITE #320,0003088937-0003-4,13,NaN,5539,W SUNSET BLVD,NaN
1973,91505-4316,NaN,NaN,BURBANK,VUBIQUITY ENTERTAINMENT CORP,NaN,NaN,3900 ALAMEDA 91505-4316,3900 W ALAMEDA AVENUE FLOOR #17,2020-09-30T00:00:00.000,NaN,2008-05-19T00:00:00.000,NaN,0002468996-0001-0,0,NaN,3900,W ALAMEDA AVENUE,FLOOR #17
1974,90071-1602,NaN,NaN,LOS ANGELES,"ODAKYU ELECTRIC RAILWAY CO., LTD.",NaN,NaN,445 FIGUEROA 90071,445 S FIGUEROA STREET UNIT #3151,2020-09-30T00:00:00.000,NaN,2020-03-01T00:00:00.000,NaN,0003182423-0001-7,14,NaN,445,S FIGUEROA STREET,UNIT #3151
1975,90004-6078,NaN,NaN,LOS ANGELES,JINOA INTERNATIONAL INC,NaN,NaN,167 HOOVER 90004,167 S HOOVER STREET,2020-09-30T00:00:00.000,NaN,2019-07-23T00:00:00.000,NaN,0003144873-0001-7,13,NaN,167,S HOOVER STREET,NaN


In [9]:
#creates address column
df['Address'] = df['number']+' '+df['street']+', '+df['city']+', '+'CA'+', '+df['zip_code']
df

,zip_code,location,mailing_city,city,business_name,naics,primary_naics_description,location_description,street_address,location_end_date,mailing_zip_code,location_start_date,mailing_address,location_account,council_district,dba_name,number,street,apt_unit,Address
0,90048-4307,"{'latitude': '34.0719', 'human_address': '{""ad...",WEST HOLLYWOOD,LOS ANGELES,M & S VALET PARKING SERVICES LLC,812930,Parking lots & garages,8022 3RD 90048-4307,8022 W 3RD STREET,2020-03-15T00:00:00.000,90069-4195,2014-04-11T00:00:00.000,1017 N LA CIENEGA BLVD SUITE #200,0002045255-0034-3,5,NaN,8022,W 3RD STREET,NaN,"8022 W 3RD STREET, LOS ANGELES, CA, 90048-4307"
1,90010-2830,"{'latitude': '34.0624', 'human_address': '{""ad...",THOUSAND OAKS,LOS ANGELES,WILLIAM JAMES OSBORNE,541100,Legal services,3731 WILSHIRE 90010-2830,3731 WILSHIRE BLVD #514,2020-03-15T00:00:00.000,91320-2151,1998-01-05T00:00:00.000,3155 OLD CONEJO ROAD,0000813748-0001-9,10,NaN,3731,WILSHIRE BLVD,#514,"3731 WILSHIRE BLVD , LOS ANGELES, CA, 900..."
2,90004-0000,NaN,NaN,LOS ANGELES,M & S VALET PARKING SERVICES LLC,NaN,NaN,670 LUCERNE 90004-0000,670 N LUCERNE BLVD,2020-03-15T00:00:00.000,NaN,2016-05-05T00:00:00.000,NaN,0002045255-0059-9,4,NaN,670,N LUCERNE BLVD,NaN,"670 N LUCERNE BLVD, LOS ANGELES, CA, 90004-0000"
3,90019-6901,NaN,NaN,LOS ANGELES,M & S VALET PARKING SERVICES LLC,NaN,NaN,1048 LA BREA 90019-6901,1048 S LA BREA AVENUE,2020-03-15T00:00:00.000,NaN,2016-05-26T00:00:00.000,NaN,0002045255-0060-2,10,NaN,1048,S LA BREA AVENUE,NaN,"1048 S LA BREA AVENUE, LOS ANGELES, CA, 90019..."
4,90048-4309,"{'latitude': '34.0721', 'human_address': '{""ad...",WEST HOLLYWOOD,LOS ANGELES,M & S VALET PARKING SERVICES LLC,812930,Parking lots & garages,8108 3RD 90048-4309,8108 W 3RD STREET,2020-03-15T00:00:00.000,90069-4195,2009-12-01T00:00:00.000,1017 N LA CIENEGA BLVD SUITE #200,0002045255-0006-8,5,NaN,8108,W 3RD STREET,NaN,"8108 W 3RD STREET, LOS ANGELES, CA, 90048-4309"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972,90028-8521,"{'latitude': '34.0983', 'human_address': '{""ad...",DIAMOND BAR,HOLLYWOOD,"OHB RESTAURANT, LLC",722110,Full-service restaurants,5539 SUNSET 90028-8521,5539 W SUNSET BLVD,2020-09-30T00:00:00.000,91765-5499,2019-01-01T00:00:00.000,21700 COPLEY DRIVE SUITE #320,0003088937-0003-4,13,NaN,5539,W SUNSET BLVD,NaN,"5539 W SUNSET BLVD, HOLLYWOOD, CA, 90028-8521"
1973,91505-4316,NaN,NaN,BURBANK,VUBIQUITY ENTERTAINMENT CORP,NaN,NaN,3900 ALAMEDA 91505-4316,3900 W ALAMEDA AVENUE FLOOR #17,2020-09-30T00:00:00.000,NaN,2008-05-19T00:00:00.000,NaN,0002468996-0001-0,0,NaN,3900,W ALAMEDA AVENUE,FLOOR #17,"3900 W ALAMEDA AVENUE , BURBANK, CA, 91505-..."
1974,90071-1602,NaN,NaN,LOS ANGELES,"ODAKYU ELECTRIC RAILWAY CO., LTD.",NaN,NaN,445 FIGUEROA 90071,445 S FIGUEROA STREET UNIT #3151,2020-09-30T00:00:00.000,NaN,2020-03-01T00:00:00.000,NaN,0003182423-0001-7,14,NaN,445,S FIGUEROA STREET,UNIT #3151,"445 S FIGUEROA STREET , LOS ANGELES, CA, 9007..."
1975,90004-6078,NaN,NaN,LOS ANGELES,JINOA INTERNATIONAL INC,NaN,NaN,167 HOOVER 90004,167 S HOOVER STREET,2020-09-30T00:00:00.000,NaN,2019-07-23T00:00:00.000,NaN,0003144873-0001-7,13,NaN,167,S HOOVER STREET,NaN,"167 S HOOVER STREET, LOS ANGELES, CA, 90004-6078"


In [10]:
df

,zip_code,location,mailing_city,city,business_name,naics,primary_naics_description,location_description,street_address,location_end_date,mailing_zip_code,location_start_date,mailing_address,location_account,council_district,dba_name,number,street,apt_unit,Address
0,90048-4307,"{'latitude': '34.0719', 'human_address': '{""ad...",WEST HOLLYWOOD,LOS ANGELES,M & S VALET PARKING SERVICES LLC,812930,Parking lots & garages,8022 3RD 90048-4307,8022 W 3RD STREET,2020-03-15T00:00:00.000,90069-4195,2014-04-11T00:00:00.000,1017 N LA CIENEGA BLVD SUITE #200,0002045255-0034-3,5,NaN,8022,W 3RD STREET,NaN,"8022 W 3RD STREET, LOS ANGELES, CA, 90048-4307"
1,90010-2830,"{'latitude': '34.0624', 'human_address': '{""ad...",THOUSAND OAKS,LOS ANGELES,WILLIAM JAMES OSBORNE,541100,Legal services,3731 WILSHIRE 90010-2830,3731 WILSHIRE BLVD #514,2020-03-15T00:00:00.000,91320-2151,1998-01-05T00:00:00.000,3155 OLD CONEJO ROAD,0000813748-0001-9,10,NaN,3731,WILSHIRE BLVD,#514,"3731 WILSHIRE BLVD , LOS ANGELES, CA, 900..."
2,90004-0000,NaN,NaN,LOS ANGELES,M & S VALET PARKING SERVICES LLC,NaN,NaN,670 LUCERNE 90004-0000,670 N LUCERNE BLVD,2020-03-15T00:00:00.000,NaN,2016-05-05T00:00:00.000,NaN,0002045255-0059-9,4,NaN,670,N LUCERNE BLVD,NaN,"670 N LUCERNE BLVD, LOS ANGELES, CA, 90004-0000"
3,90019-6901,NaN,NaN,LOS ANGELES,M & S VALET PARKING SERVICES LLC,NaN,NaN,1048 LA BREA 90019-6901,1048 S LA BREA AVENUE,2020-03-15T00:00:00.000,NaN,2016-05-26T00:00:00.000,NaN,0002045255-0060-2,10,NaN,1048,S LA BREA AVENUE,NaN,"1048 S LA BREA AVENUE, LOS ANGELES, CA, 90019..."
4,90048-4309,"{'latitude': '34.0721', 'human_address': '{""ad...",WEST HOLLYWOOD,LOS ANGELES,M & S VALET PARKING SERVICES LLC,812930,Parking lots & garages,8108 3RD 90048-4309,8108 W 3RD STREET,2020-03-15T00:00:00.000,90069-4195,2009-12-01T00:00:00.000,1017 N LA CIENEGA BLVD SUITE #200,0002045255-0006-8,5,NaN,8108,W 3RD STREET,NaN,"8108 W 3RD STREET, LOS ANGELES, CA, 90048-4309"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972,90028-8521,"{'latitude': '34.0983', 'human_address': '{""ad...",DIAMOND BAR,HOLLYWOOD,"OHB RESTAURANT, LLC",722110,Full-service restaurants,5539 SUNSET 90028-8521,5539 W SUNSET BLVD,2020-09-30T00:00:00.000,91765-5499,2019-01-01T00:00:00.000,21700 COPLEY DRIVE SUITE #320,0003088937-0003-4,13,NaN,5539,W SUNSET BLVD,NaN,"5539 W SUNSET BLVD, HOLLYWOOD, CA, 90028-8521"
1973,91505-4316,NaN,NaN,BURBANK,VUBIQUITY ENTERTAINMENT CORP,NaN,NaN,3900 ALAMEDA 91505-4316,3900 W ALAMEDA AVENUE FLOOR #17,2020-09-30T00:00:00.000,NaN,2008-05-19T00:00:00.000,NaN,0002468996-0001-0,0,NaN,3900,W ALAMEDA AVENUE,FLOOR #17,"3900 W ALAMEDA AVENUE , BURBANK, CA, 91505-..."
1974,90071-1602,NaN,NaN,LOS ANGELES,"ODAKYU ELECTRIC RAILWAY CO., LTD.",NaN,NaN,445 FIGUEROA 90071,445 S FIGUEROA STREET UNIT #3151,2020-09-30T00:00:00.000,NaN,2020-03-01T00:00:00.000,NaN,0003182423-0001-7,14,NaN,445,S FIGUEROA STREET,UNIT #3151,"445 S FIGUEROA STREET , LOS ANGELES, CA, 9007..."
1975,90004-6078,NaN,NaN,LOS ANGELES,JINOA INTERNATIONAL INC,NaN,NaN,167 HOOVER 90004,167 S HOOVER STREET,2020-09-30T00:00:00.000,NaN,2019-07-23T00:00:00.000,NaN,0003144873-0001-7,13,NaN,167,S HOOVER STREET,NaN,"167 S HOOVER STREET, LOS ANGELES, CA, 90004-6078"


In [11]:
#drops unnecessary columns
df_fin = df.drop(columns=['location', 'mailing_city', 'naics','location_description','street_address','mailing_zip_code','location_account', 'council_district','dba_name', 'apt_unit','number','street','apt_unit'])
df_fin

,zip_code,city,business_name,primary_naics_description,location_end_date,location_start_date,mailing_address,Address
0,90048-4307,LOS ANGELES,M & S VALET PARKING SERVICES LLC,Parking lots & garages,2020-03-15T00:00:00.000,2014-04-11T00:00:00.000,1017 N LA CIENEGA BLVD SUITE #200,"8022 W 3RD STREET, LOS ANGELES, CA, 90048-4307"
1,90010-2830,LOS ANGELES,WILLIAM JAMES OSBORNE,Legal services,2020-03-15T00:00:00.000,1998-01-05T00:00:00.000,3155 OLD CONEJO ROAD,"3731 WILSHIRE BLVD , LOS ANGELES, CA, 900..."
2,90004-0000,LOS ANGELES,M & S VALET PARKING SERVICES LLC,NaN,2020-03-15T00:00:00.000,2016-05-05T00:00:00.000,NaN,"670 N LUCERNE BLVD, LOS ANGELES, CA, 90004-0000"
3,90019-6901,LOS ANGELES,M & S VALET PARKING SERVICES LLC,NaN,2020-03-15T00:00:00.000,2016-05-26T00:00:00.000,NaN,"1048 S LA BREA AVENUE, LOS ANGELES, CA, 90019..."
4,90048-4309,LOS ANGELES,M & S VALET PARKING SERVICES LLC,Parking lots & garages,2020-03-15T00:00:00.000,2009-12-01T00:00:00.000,1017 N LA CIENEGA BLVD SUITE #200,"8108 W 3RD STREET, LOS ANGELES, CA, 90048-4309"
...,...,...,...,...,...,...,...,...
1972,90028-8521,HOLLYWOOD,"OHB RESTAURANT, LLC",Full-service restaurants,2020-09-30T00:00:00.000,2019-01-01T00:00:00.000,21700 COPLEY DRIVE SUITE #320,"5539 W SUNSET BLVD, HOLLYWOOD, CA, 90028-8521"
1973,91505-4316,BURBANK,VUBIQUITY ENTERTAINMENT CORP,NaN,2020-09-30T00:00:00.000,2008-05-19T00:00:00.000,NaN,"3900 W ALAMEDA AVENUE , BURBANK, CA, 91505-..."
1974,90071-1602,LOS ANGELES,"ODAKYU ELECTRIC RAILWAY CO., LTD.",NaN,2020-09-30T00:00:00.000,2020-03-01T00:00:00.000,NaN,"445 S FIGUEROA STREET , LOS ANGELES, CA, 9007..."
1975,90004-6078,LOS ANGELES,JINOA INTERNATIONAL INC,NaN,2020-09-30T00:00:00.000,2019-07-23T00:00:00.000,NaN,"167 S HOOVER STREET, LOS ANGELES, CA, 90004-6078"


In [12]:
#creates sample to test code with all 1977 closed businesses the normal geocode will take 30 minutes
df_sample = df_fin.sample(n=20, random_state=8)
df_sample

,zip_code,city,business_name,primary_naics_description,location_end_date,location_start_date,mailing_address,Address
439,90048-3664,LOS ANGELES,WENDY SUNG,NaN,2020-04-24T00:00:00.000,2018-11-01T00:00:00.000,NaN,"104 S HAYWORTH AVENUE , LOS ANGELES, CA, 90..."
195,91311-7916,CHATSWORTH,THE EDGE MARTIAL ARTS INC,NaN,2020-03-30T00:00:00.000,2016-10-01T00:00:00.000,16622 SOLEDAD CANYON ROAD,"10010 LARWIN AVENUE, CHATSWORTH, CA, 91311-..."
1793,90640-2170,MONTEBELLO,"EYE PEOPLE OPTOMETRY, INC.",NaN,2020-09-08T00:00:00.000,2018-03-12T00:00:00.000,NaN,"2156 MONTEBELLO TOWN CENTER, MONTEBELLO, CA..."
1182,90012-2398,LOS ANGELES,JIANWEI HUONG,NaN,2020-06-30T00:00:00.000,2019-03-08T00:00:00.000,NaN,"857 YALE STREET , LOS ANGELES, CA, 90012-..."
573,91324-2744,NORTHRIDGE,SOPHIRA SERVICES INC,NaN,2020-05-08T00:00:00.000,2013-09-01T00:00:00.000,NaN,"9332 TAMPA AVENUE, NORTHRIDGE, CA, 91324-2744"
1177,91401-2032,VAN NUYS,ELOISA AMAYA,Janitorial services,2020-06-30T00:00:00.000,2008-09-02T00:00:00.000,NaN,"13238 VICTORY BLVD, VAN NUYS, CA, 91401-2032"
1840,92801-5373,ANAHEIM,LISTEN FILM LLC,NaN,2020-09-16T00:00:00.000,2012-03-06T00:00:00.000,NaN,"1950 W CORPORATE WAY , ANAHEIM, CA, 92801-5373"
1466,91780-3920,TEMPLE CITY,NEUTEK CORPORATION,NaN,2020-07-31T00:00:00.000,2018-05-01T00:00:00.000,NaN,"9856 LA ROSA DRIVE, TEMPLE CITY, CA, 91780-..."
61,91604-2017,STUDIO CITY,SUMBODY FITNESS LLC,Spectator sports (including professional sport...,2020-03-16T00:00:00.000,2015-03-20T00:00:00.000,1223 WILSHIRE BLVD #202,"4070 LAUREL CANYON BLVD, STUDIO CITY, CA, 916..."
223,90020-3805,LOS ANGELES,JISU HAN,NaN,2020-03-31T00:00:00.000,2019-10-01T00:00:00.000,1101 S HARVARD BLVD APT #508,"324 S WESTERN AVENUE , LOS ANGELES, CA, 900..."


In [13]:
#creates empty lat and long columns
df_sample['lat'] = pd.Series(dtype='float')
df_sample['lon'] = pd.Series(dtype='float')
df_sample

,zip_code,city,business_name,primary_naics_description,location_end_date,location_start_date,mailing_address,Address,lat,lon
439,90048-3664,LOS ANGELES,WENDY SUNG,NaN,2020-04-24T00:00:00.000,2018-11-01T00:00:00.000,NaN,"104 S HAYWORTH AVENUE , LOS ANGELES, CA, 90...",NaN,NaN
195,91311-7916,CHATSWORTH,THE EDGE MARTIAL ARTS INC,NaN,2020-03-30T00:00:00.000,2016-10-01T00:00:00.000,16622 SOLEDAD CANYON ROAD,"10010 LARWIN AVENUE, CHATSWORTH, CA, 91311-...",NaN,NaN
1793,90640-2170,MONTEBELLO,"EYE PEOPLE OPTOMETRY, INC.",NaN,2020-09-08T00:00:00.000,2018-03-12T00:00:00.000,NaN,"2156 MONTEBELLO TOWN CENTER, MONTEBELLO, CA...",NaN,NaN
1182,90012-2398,LOS ANGELES,JIANWEI HUONG,NaN,2020-06-30T00:00:00.000,2019-03-08T00:00:00.000,NaN,"857 YALE STREET , LOS ANGELES, CA, 90012-...",NaN,NaN
573,91324-2744,NORTHRIDGE,SOPHIRA SERVICES INC,NaN,2020-05-08T00:00:00.000,2013-09-01T00:00:00.000,NaN,"9332 TAMPA AVENUE, NORTHRIDGE, CA, 91324-2744",NaN,NaN
1177,91401-2032,VAN NUYS,ELOISA AMAYA,Janitorial services,2020-06-30T00:00:00.000,2008-09-02T00:00:00.000,NaN,"13238 VICTORY BLVD, VAN NUYS, CA, 91401-2032",NaN,NaN
1840,92801-5373,ANAHEIM,LISTEN FILM LLC,NaN,2020-09-16T00:00:00.000,2012-03-06T00:00:00.000,NaN,"1950 W CORPORATE WAY , ANAHEIM, CA, 92801-5373",NaN,NaN
1466,91780-3920,TEMPLE CITY,NEUTEK CORPORATION,NaN,2020-07-31T00:00:00.000,2018-05-01T00:00:00.000,NaN,"9856 LA ROSA DRIVE, TEMPLE CITY, CA, 91780-...",NaN,NaN
61,91604-2017,STUDIO CITY,SUMBODY FITNESS LLC,Spectator sports (including professional sport...,2020-03-16T00:00:00.000,2015-03-20T00:00:00.000,1223 WILSHIRE BLVD #202,"4070 LAUREL CANYON BLVD, STUDIO CITY, CA, 916...",NaN,NaN
223,90020-3805,LOS ANGELES,JISU HAN,NaN,2020-03-31T00:00:00.000,2019-10-01T00:00:00.000,1101 S HARVARD BLVD APT #508,"324 S WESTERN AVENUE , LOS ANGELES, CA, 900...",NaN,NaN


In [ ]:
# loop through list and add to dataframe with lat/lon's Yoh example
for index, row in df_sample.iterrows():

    # identify the address column
    address = row['Address']
    
    try:
        
        # geocode it
        geocoded_address = ox.geocoder.geocode(address)

        # add it to the dataframe
        df_sample.at[index,'lat']=geocoded_address[0]
        df_sample.at[index,'lon']=geocoded_address[1]

        # print the output
        print(address + ' geocoded to ' + str(geocoded_address[0])+' '+str(geocoded_address[1]))        

    except:
        print('Could not geocode '+ address)
        pass

104  S HAYWORTH AVENUE   , LOS ANGELES, CA, 90048-3664 geocoded to 34.074097306122454 -118.36251863265306
10010    LARWIN AVENUE, CHATSWORTH, CA, 91311-7916 geocoded to 34.254377311110325 -118.61756758753708


In [ ]:
#sets one second time limit to geocoder
geocoder = RateLimiter(Nominatim(user_agent='name').geocode, min_delay_seconds=1)


In [ ]:
#geocodes sample addressses + another way to geocode
df_sample['locationn']=df_sample['Address'].apply(geocoder)

# add lat and lon
df_sample['Latitude'] = df_sample['locationn'].apply(lambda loc: loc.latitude if loc else None)
df_sample['Longitude'] = df_sample['locationn'].apply(lambda loc: loc.longitude if loc else None)

df_sample

In [ ]:
#simple plot on plotly
fig = px.scatter_mapbox(df_sample,
                        lat='lat',
                        lon='lon',
                        mapbox_style="stamen-terrain")
fig.update_layout(mapbox_style="carto-darkmatter")
fig.show()

In [ ]:
#import kepler gl 
from keplergl import KeplerGl

In [ ]:
#drop and simplify data to map to Kepler
df_map = df_sample.drop(columns=['zip_code', 'mailing_address', 'locationn', 'Latitude','Longitude'])
df_map
#export to csv
df_map.to_csv('data/df.csv', encoding='utf-8')
df_map

In [ ]:
#convert lat and lon to number
df_map['lat'] = pd.to_numeric(df_map['lat'])
df_map['lon'] = pd.to_numeric(df_map['lon'])
df_map

In [ ]:
#there are 6 addresses without Lat, Lon and will not be mapped
df_map.isna().sum()

In [ ]:
#delete locations with null lat and lon. you can choose lat or lon in this case
df_map=df_map[df_map['lat'].notna()]
df_map

In [ ]:
map = KeplerGl(height=600,width=800)
map


In [ ]:
map.add_data(data=df_map, name='Address')

#not sure why this is not displaying, but Kepler isn't ideal for mapping multiple layers
with open('data/ct2010.geojson', 'r') as f:
    geojson = f.read()
map.add_data(data=geojson,name="geojson")

In [ ]:
#load geojson and keep necessary columns
svi = gpd.read_file('data/si.json')
svi=svi[['FIPS','RPL_THEMES','geometry']]
svi.head()
svi.info()

In [ ]:
#load csv to join data
svi_data = pd.read_csv('data/svi_final.csv')
svi_data=svi_data[['FIPS', 'RPL_THEMES']]

svi_data.info()

In [ ]:
#convert fips to string 
svi_data['FIPS'] = svi_data['FIPS'].apply(str)
svi_data.info()

In [ ]:
#add leading zero to join with json data
svi_data['FIPS'] = svi_data['FIPS'].str.zfill(11)
svi_data.head()

In [ ]:
#add empty folium map 
m = folium.Map(location=[34.051850, -118.243322], tiles=None)

folium.raster_layers.TileLayer(
    location=[34.051850, -118.243322],
    tiles='https://api.mapbox.com/styles/v1/cgiamarino/ckh3zcblc09pp19n0adewnboh/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoiY2dpYW1hcmlubyIsImEiOiJjanAyd2hwNWUwYnJ3M3dzMHF4MnJkZ29uIn0.7tLnplw65tLmywvvIPGSKA',
    attr='Mapbox',
    API_key='pk.eyJ1IjoiY2dpYW1hcmlubyIsImEiOiJjanAyd2hwNWUwYnJ3M3dzMHF4MnJkZ29uIn0.7tLnplw65tLmywvvIPGSKA',
    name='Monochrome',
    zoom_start=9  # Limited levels of zoom for free Mapbox tiles.
).add_to(m)


#create choropleth
folium.Choropleth(
    geo_data=svi,
    name='SVI',
    data=svi_data,
    popup=svi_data.RPL_THEMES,
    columns=['FIPS', 'RPL_THEMES'],
    fill_color='PuRd',
    key_on='feature.properties.FIPS',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Social Vulnerability Index (SVI)'
).add_to(m)

#create feature group so that business layer can be added to drop down
f2=folium.FeatureGroup(name='Business Closures').add_to(m)

# add the businesses to the feature group
for index, row in df_map.iterrows():
    folium.Circle(
        radius=30,
        location=[row.lat,row.lon], 
        popup=row.business_name+ ' '+row.Address, 
        tooltip=row.business_name,
        overlay=False).add_to(f2)



m.save('svi_business.html')

folium.LayerControl(position='topright', collapsed=True, autoZIndex=True).add_to(m)


m